# 3. Customized model

One of the ideas of PiNN is that different atomic neural networks shares similar building blocks.

PiNN's abstraction of layers allows us to construct new network structures from those building blocks.  
It also makes it easier to design novel layers that works along with existing ones.

In this notebook, we will build a customized model by combining PiNN's Pi blocks with Behler's  
element specific neural network. In this way, Pi blocks serves as a "learnable" feature generator.

In [ ]:
import pinn.layers as l
import pinn.filters as f

## Defining a custom model function

In [ ]:
def hybrid_network(tensors):
    filters = {
        f.sparsify()
        f.atomic_onehot()
        f.atomic_dress(dress={1:10,6:3})
        f.naive_nl(4)(tensors)
        f.symm_func()(tensors)
        f.pi_basis()(tensors)}
    for f in filters:
        f(tensors)
    # Define the filters, Note that you can include a pre_level option
    # to optimize the input pipline.
    
    
    # PiNN blocks copied from pinn.networks.
    # After several blocks, the updated atomic features are stored in nodes[1] .
    
    # BPNN blocks copied from pinn.networks.
    
    # Generate the final predictions (total energy).
    return pred

## Train and evaluation

After that, the model can be readily trained.

In [ ]:
params = {'network':hybrid_network}
model = potential_model(params)
# Dataset and training specs
datasets = load_QM9_dataset('dsgn16/*', split_ratio={'train':8, 'test':2})
train = lambda: datasets['train']().repeat().batch(100)
test = lambda: datasets['test']().repeat().batch(10)
train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=100)
# Run the training
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## Notes
You can also save the parameters of custom models for later use, however,  
there's currently no way to recover a model if you lost the original function.